In [ ]:
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import re
import datetime
import os
import string



# VARIABLES TO ADJUST

In [ ]:
now= datetime.datetime.now()
dt_format = '%Y-%m-%d'
strToday = now.strftime(dt_format)
strToday

In [ ]:
customer_name = 'soriana' # will be placed in the final file name
client_file_name ='soriana.csv' 
filter_file_name = 'mp_competencia.xlsx'

#Save and Read paths
path = f'C:/Users/IvanMinauro/Documents/fuzzy/fuzzy'
path_base = f'C:/Users/IvanMinauro/Documents/fuzzy/fuzzy/client_data/'
path_comparar = f'C:/Users/IvanMinauro/Documents/fuzzy/fuzzy/competitors_csv/'
final_file_name = f'C:/Users/IvanMinauro/Documents/fuzzy/fuzzy/match_comparisson_{customer_name}_{strToday}.csv'

# FUNCTIONS

In [ ]:
#
import pandas as pd
import glob
import re
from datetime import timedelta
from datetime import date

#Comment test git
LIST_COLUMN_ORDER = ['Date', 'Canal', 'Category', 'Subcategory', 'Subcategory2', 'Subcategory3', 'Marca',
                     'Modelo', 'SKU', 'UPC', 'Item', 'Item Characteristics', 'URL SKU', 'Image', 'Price',
                     'Sale Price', 'Shipment Cost', 'Sales Flag', 'Store ID', 'Store Name',
                     'Store Address', 'Stock', 'UPC WM2', 'Final Price', 'UPC WM', 'COMP']

LIST_COLUMN_ORDER_SHORT = ['Date', 'Canal', 'Category', 'Subcategory', 'Subcategory2', 'Subcategory3', 'Marca',
                     'Modelo', 'SKU', 'UPC', 'Item', 'Item Characteristics', 'URL SKU', 'Image', 'Price',
                     'Sale Price', 'Shipment Cost', 'Sales Flag', 'Store ID', 'Store Name',
                     'Store Address', 'Stock', 'UPC WM', 'Final Price']

LIST_COLUMN_ORDER_M = ['Date', 'Canal', 'Category', 'Subcategory', 'Subcategory2', 'Subcategory3', 'Marca',
                      'Modelo', 'SKU', 'UPC', 'Item', 'Item Characteristics', 'URL SKU', 'Image', 'Price',
                      'Sale Price', 'Shipment Cost', 'Sales Flag', 'Store ID', 'Store Name',
                      'Store Address', 'Stock', 'UPC WM2', 'Final Price', 'UPC WM', 'COMP',
                      'GR', 'CAT C', 'SUB C', 'MARCA C'	, 'Precio Gramo', 'MARCA 2', 'Congelado']

COMPETITORS_FLOAT_COLUMNS = ['Price', 'Final Price', 'Sale Price']
COMPETITORS_STRING_COLUMNS = ['UPC', 'EAN', 'UPC WM', 'UPC WM2']
COMPARISON_STRING_COLUMNS = ['upc_wm2_client', 'match', 'upc_wm2_competitor']

#Start Data Frame Utils

#This section is only for functions that work on columns of a Data Frame
#   object. To call them use the class method apply().
def create_upc_wm(strUPC, strChannel = ''):
    
    if strUPC.isdigit():
        if 'Walmart' in strChannel or 'walmart' in strChannel:
            pass
        else:
            if len(strUPC) > 7:
                strUPC = strUPC[:-1]
    
        while len(strUPC) < 16:
                strUPC = '0' + strUPC
                
    return strUPC

def select_upc_wm2(strUPCWM_Competitors, strUPCWM_Comparison):
    if pd.isna(strUPCWM_Comparison):
        return strUPCWM_Competitors
    else:
        return strUPCWM_Comparison

def get_weights(strItem, regPattern):
    #Buscar optimizar esta sección

    lstGram_Names = ['g', 'gr', 'gramo', 'gramos', 'grms', 'grm']
    lstFuzzy_Words = ['granos']

    for strFuzzy_Word in lstFuzzy_Words:
        strItem = strItem.replace(strFuzzy_Word, "")

    if any(strGram_Name in strItem for strGram_Name in lstGram_Names):
        strExtraction = strItem.replace(' ',  '')

        if regPattern.search(strExtraction):
            strExtraction = regPattern.search(strExtraction)
            strExtraction = strExtraction.group(1)

            strExtraction = strExtraction.replace('g', '')
            return strExtraction
        else:
            return '0'
    elif 'bolzalza' in strItem:
        return 'Bolzalza'
    else:
        return '0'

def get_weights_kilograms(strItem, regPattern):
    #Buscar optimizar esta sección

    lstGram_Names = ['kg', 'kgr', 'kilogramo', 'kilo']

    if any(strGram_Name in strItem for strGram_Name in lstGram_Names):
        strExtraction = strItem.replace(' ',  '')

        if regPattern.search(strExtraction):
            strExtraction = regPattern.search(strExtraction)
            strExtraction = strExtraction.group(1)

            strExtraction = strExtraction.replace('k', '')
            fltExtraction = float(strExtraction) * 1000

            return str(fltExtraction)
        else:
            return '0'
    else:
        return '0'

def calculate_discount(fltPrice, fltFinal_Price):
    return 1 - (fltFinal_Price / fltPrice)

def calculate_price_per_kg(fltFinal_Price, strGram, boolIts_Pack = False):
    if strGram != 'Bolzalza' and strGram != '0' and boolIts_Pack == False:
        fltGram = float(strGram)
        fltPrice_Gram = fltFinal_Price / fltGram
        fltPrice_Gram = fltPrice_Gram * 1000

        return fltPrice_Gram
    else:
        return 0

def determine_if_pack(strItem):
    lstPieces_Names = ['piezas', 'pzs', 'pzas']

    if any(strPieces_Name in strItem for strPieces_Name in lstPieces_Names):
        return True
    else:
        return False

#End Data Frame Utils

#Start File Util Functions
def convert_excel_to_df(strExcel_Path):
    lstSheets = pd.ExcelFile(strExcel_Path).sheet_names

    dfConcat_Excel = pd.concat([pd.read_excel(strExcel_Path, sheet_name = strSheet_Name) for strSheet_Name
                           in lstSheets], ignore_index = True)

    return dfConcat_Excel

def consolidate_competitors_df(strPath_Read, lstItem_Words = []):
    if len(strPath_Read) < 3:
        raise ValueError('Path is too short for a valid file location.')

    lstFilenames = glob.glob(strPath_Read + "\*.csv") #Toma de ese path los archivos que tengan como extención csv

    lstDFs_Concat = []
    for strFilename in lstFilenames: #Nombre del los csv
        print(strFilename)
        dfAux = pd.read_csv(strFilename, dtype = str) #dataframe leido por iteracion

        lstColumns = dfAux.columns.to_list() #Nombres de las columnas
        dictNew_Columns = homogonize_column_names(lstColumns)
        dfAux.rename(columns = dictNew_Columns, inplace = True)

        lstDFs_Concat.append(dfAux) #añadir a lista de dataframes

    dfConsolidated_Competitors = pd.concat(lstDFs_Concat) #Concatenar lista de dataframe a un solo dataframe

    if len(lstItem_Words) > 0:
        dfConsolidated_Competitors = dfConsolidated_Competitors[
                                        dfConsolidated_Competitors['Item'].str.contains('|'.join(lstItem_Words))
                                        ].reset_index(drop = True) #Filtrar por palabras

    dfConsolidated_Competitors.drop(columns=['UPC WM'], inplace = True) #Eliminar UPC WN
    dfConsolidated_Competitors['UPC'] = dfConsolidated_Competitors['UPC'].str.replace('\.0+$', '',
                                           regex = True)
    dfConsolidated_Competitors['UPC'] = dfConsolidated_Competitors['UPC'].fillna('')
    dfConsolidated_Competitors['UPC'] = dfConsolidated_Competitors['UPC'].astype(str)
    dfConsolidated_Competitors['Canal'] = dfConsolidated_Competitors['Canal'].astype(str)
    dfConsolidated_Competitors['UPC WM'] = dfConsolidated_Competitors.apply(lambda row:
                                             create_upc_wm(row['UPC'], row['Canal']), axis = 1)
    dfConsolidated_Competitors['UPC WM2'] = dfConsolidated_Competitors['UPC WM']


    dfConsolidated_Competitors[COMPETITORS_FLOAT_COLUMNS] = dfConsolidated_Competitors[COMPETITORS_FLOAT_COLUMNS].replace(
                                                                {'\$': '',
                                                                 ',': '',
                                                                 r'\[': '',
                                                                 r'\]': '',
                                                                 'Price': '0'}, regex = True) #Reemplazar ciertos caracteres

    #dfConsolidated_Competitors[COMPETITORS_FLOAT_COLUMNS] = dfConsolidated_Competitors[COMPETITORS_FLOAT_COLUMNS].astype(float)

    dfConsolidated_Competitors['COMP'] = ''

    dfConsolidated_Competitors = dfConsolidated_Competitors[LIST_COLUMN_ORDER] #Ordenar dataframe

    dfConsolidated_Competitors = dfConsolidated_Competitors[(dfConsolidated_Competitors['Price'].notna()) |
                                                            (dfConsolidated_Competitors['Price'] != '0')] #Filtrar por precio vacio

    return dfConsolidated_Competitors

def get_comparison_df(strPath_Comparison_File, strSheet_Name):
    dictComparison_Dtypes = {strColumn_Name: 'str' for strColumn_Name in COMPARISON_STRING_COLUMNS}

    dfComparison = pd.read_excel(strPath_Comparison_File, sheet_name = strSheet_Name,
                                dtype = dictComparison_Dtypes)

    dfComparison = dfComparison[['upc_wm2_client', 'match', 'upc_wm2_competitor']]
    dfComparison = dfComparison.dropna(subset = ['upc_wm2_competitor'])
    dfComparison.drop_duplicates(inplace = True)

    return dfComparison

def replace_upc_wm2(dfCompetitors, dfComparison):
    dfCompetitors = dfCompetitors.merge(dfComparison, how = 'left' ,left_on = 'UPC WM2', right_on = 'upc_wm2_competitor')

    dfCompetitors['UPC WM'] = dfCompetitors.apply(lambda row:  select_upc_wm2(row['UPC WM2'],
                                  row['upc_wm2_client']), axis = 1)

    dfCompetitors.drop(columns = COMPARISON_STRING_COLUMNS, inplace = True)

    return dfCompetitors

def extract_weight(dfCompetitors):
    strRegex_Grams = '([0-9]+\.?[0-9]*\s?g)'
    strRegex_Kgs = '([0-9]+\.?[0-9]*\s?k)'
    lstPer_KG_Names = ['por kilo', 'el kilo', 'por kg', 'kg', 'kilo']

    regPattern_Grams = re.compile(strRegex_Grams)
    regPattern_KG = re.compile(strRegex_Kgs)

    dfCompetitors['aux_item'] = dfCompetitors['Item']
    dfCompetitors['aux_item'] = dfCompetitors['aux_item'].astype(str)
    dfCompetitors['aux_item'] = dfCompetitors['aux_item'].str.lower()
    dfCompetitors['aux_item'] = dfCompetitors['aux_item'].str.replace('-', '')

    dfCompetitors_Kilos = dfCompetitors[dfCompetitors['aux_item'].str.contains(strRegex_Kgs,
                             regex = True)].reset_index(drop = True)
    dfCompetitors_aux = dfCompetitors[~dfCompetitors['aux_item'].str.contains(strRegex_Kgs,
                            regex = True)].reset_index(drop = True)

    dfCompetitors_Kilos['Cantidad'] = dfCompetitors_Kilos.apply(lambda row: get_weights_kilograms(row['aux_item'],
                                   regPattern_KG), axis = 1, result_type = 'reduce')

    dfCompetitors_Grams = dfCompetitors_aux[dfCompetitors_aux['aux_item'].str.contains(strRegex_Grams,
                              regex = True)].reset_index(drop = True)
    dfCompetitors_aux = dfCompetitors_aux[~dfCompetitors_aux['aux_item'].str.contains(strRegex_Grams
                            )].reset_index(drop = True)

    dfCompetitors_Grams['Cantidad'] = dfCompetitors_Grams.apply(lambda row: get_weights(row['aux_item'],
                               regPattern_Grams), axis = 1, result_type = 'reduce')

    dfCompetitor_One_Kilo = dfCompetitors_aux[(dfCompetitors_aux['aux_item'].str.contains('|'.join(lstPer_KG_Names),
                                regex = True))].reset_index(drop = True)
    dfCompetitors_aux = dfCompetitors_aux[(~dfCompetitors_aux['aux_item'].str.contains('|'.join(lstPer_KG_Names),
                            regex = True))].reset_index(drop = True)

    dfCompetitor_One_Kilo['Cantidad'] = 1000

    dfCompetitors_aux['Cantidad'] = 0

    dfCompetitors = pd.concat([dfCompetitors_Kilos, dfCompetitors_Grams, dfCompetitor_One_Kilo, dfCompetitors_aux],
                        ignore_index = True)

    dfCompetitors['Unidad'] = 'gr'

    dfCompetitors.drop(columns = ['aux_item'], inplace = True)

    return dfCompetitors

def extract_ml(dfCompetitors):
    reMililiters = re.compile(r'[0-9]+\.?[0-9]*ml', re.IGNORECASE)

    lstIndexs = [j for j in range(len(dfCompetitors))]
    dfCompetitors['Cantidad'] = list(map(lambda i: "".join(reMililiters.findall(dfCompetitors.Item[i].replace(" ml", "ml").replace(" - ", "")))[:-2] , lstIndexs))

    dfCompetitors.loc[dfCompetitors.Cantidad == "", 'Cantidad'] = 0
    dfCompetitors.Cantidad = dfCompetitors.Cantidad.astype(float)
    dfCompetitors['Unidad'] = 'ml'

    return dfCompetitors

def extract_quantities_and_units(dfCompetitors):
    dfCompetitors = extract_ml(dfCompetitors)

    dfCompetitors_Ml =  dfCompetitors[dfCompetitors['Cantidad'] != 0].reset_index(drop = True)
    dfCompetitors_Aux = dfCompetitors[dfCompetitors['Cantidad'] == 0].reset_index(drop = True)

    dfCompetitors_Aux = extract_weight(dfCompetitors_Aux)

    return pd.concat([dfCompetitors_Ml, dfCompetitors_Aux], ignore_index = True)

def compare_rows(dfTo_Compare, intPosition_Fst_Row, intPosition_Sec_Row):
    lstColumn_Names = dfTo_Compare.columns.to_list()

    dicResult = {}
    for strColumn_name in lstColumn_Names:
        rowFirst_Value = dfTo_Compare.loc[intPosition_Fst_Row, strColumn_name]
        rowSecond_Value = dfTo_Compare.loc[intPosition_Fst_Row, strColumn_name]

        if pd.isna(rowFirst_Value) and pd.isna(rowSecond_Value):
            dicResult[strColumn_name] = True

        else:
            dicResult[strColumn_name] = rowFirst_Value == rowSecond_Value

    return dicResult

def homogonize_column_names(lstColumns):
    setColumns_Upper = ['sku', 'upc', 'url sku', 'upc wm']

    dictNew_Columns = {}
    for strColumn_Name in lstColumns:
        strNew_Column = strColumn_Name.lower().replace('_', ' ')

        if strNew_Column == 'store id':
            dictNew_Columns[strColumn_Name] = 'Store ID'
        elif strNew_Column not in setColumns_Upper:
            dictNew_Columns[strColumn_Name] = strNew_Column.title()
        else:
            dictNew_Columns[strColumn_Name] = strNew_Column.upper()

    return dictNew_Columns
#End File Util Functions

#Function get monday date
def calculate_update_date():
    now = date.today()
    monday_date = now - timedelta(days=now.weekday())

    return monday_date.isoformat()

#Function get counts Date and UPC grouping
def getCounts(dfCompetitors):
    dfGrouping = dfCompetitors.groupby(by=["Date","UPC WM"]).size().reset_index(name='counts')
    dfCompetitors = pd.merge(dfCompetitors, dfGrouping, on=["Date", "UPC WM"])
    return dfCompetitors

# DATA PROCESSING

In [ ]:
now= datetime.datetime.now()
dt_format = '%Y-%m-%d'
strToday = now.strftime(dt_format)
strToday

In [ ]:
df_base = pd.read_csv(path_base+client_file_name)
#RENAME COLUMN TO BE ITEM FOR ITEM DESCRIPTION AND UPC FOR SKU/EAN/UPC/ART ID
df_base.rename(columns={ df_base.columns[1]: "Item" }, inplace = True)
df_base.rename(columns={ df_base.columns[0]: "UPC" }, inplace = True)
df_base

In [ ]:
df_comparar = consolidate_competitors_df(path_comparar)
df_comparar

In [ ]:
df_comparar['COMP'] =df_comparar['Store ID']+" "+ df_comparar['Category']+" "+df_comparar['Marca']+" "+df_comparar['Item']
df_comparar

In [ ]:
df_comparar['Store ID'].unique()

In [ ]:
df_filter = pd.read_excel(path+filter_file_name)
df_filter

In [ ]:
df_filtered = df_filter[df_filter['flag'].isin(['x', '{all}'])]
df_filtered

In [ ]:
df_resultado_final = pd.DataFrame()  # DataFrame final para almacenar todos los resultados

# Obtener los diferentes Store ID en df_comparar
store_ids = df_comparar['Store ID'].unique()

# Función para encontrar las mejores coincidencias
def encontrar_coincidencias(item, lista_comparar, top_n=5):
    return process.extract(item, lista_comparar, limit=top_n, scorer=fuzz.token_sort_ratio)

total_productos = len(df_base)

# Iterar sobre cada Store ID
for store_id in store_ids:
    df_comparar_filtrado = df_comparar[df_comparar['Store ID'] == store_id]
    
    # Crear un DataFrame temporal para almacenar los resultados de cada Store ID
    df_resultado = df_base.copy()
    for i in range(5):  # Añadir columnas para hasta 5 sugerencias
        df_resultado[f'Tipo de Comparación {i+1}'] = ""
        df_resultado[f'UPC Sugerido {i+1}'] = ""
        df_resultado[f'Descripción de producto Sugerido {i+1}'] = ""
        df_resultado[f'Puntuación {i+1}'] = ""
        df_resultado[f'URL SKU Sugerido {i+1}'] = ""
        df_resultado[f'Imagen Sugerida {i+1}'] = ""
        df_resultado[f'Precio Final Sugerido {i+1}'] = ""

    # Cruce por UPC y comparación de texto
    for index, row in df_base.iterrows():
        producto_actual = index + 1
        print(f"------ {producto_actual} out of {total_productos} ---- {producto_actual / total_productos:.1%} ------")
        
        upc = row['UPC']
        item = row['Item']
        if upc in df_comparar_filtrado['UPC'].values:
            coincidencia_directa = df_comparar_filtrado[df_comparar_filtrado['UPC'] == upc].iloc[0]
            df_resultado.at[index, 'Tipo de Comparación 1'] = "Idéntico"
            descripcion = df_comparar_filtrado.loc[df_comparar_filtrado['UPC'] == upc, 'Item'].iloc[0]
            df_resultado.at[index, 'Descripción de producto Sugerido 1'] = descripcion
            df_resultado.at[index, 'UPC Sugerido 1'] = upc
            df_resultado.at[index, 'Puntuación 1'] = 100
            df_resultado.at[index, 'URL SKU Sugerido 1'] = coincidencia_directa['URL SKU']
            df_resultado.at[index, 'Imagen Sugerida 1'] = coincidencia_directa['Image']
            df_resultado.at[index, 'Precio Final Sugerido 1'] = coincidencia_directa['Final Price']

        else:
            coincidencias = encontrar_coincidencias(item, df_comparar_filtrado['Item'].tolist())
            for i, (desc, score) in enumerate(coincidencias):
                fila_sugerida = df_comparar_filtrado[df_comparar_filtrado['Item'] == desc].iloc[0]
                df_resultado.at[index, f'Tipo de Comparación {i+1}'] = "Sugerido"
                upc_sugerido = df_comparar_filtrado[df_comparar_filtrado['Item'] == desc]['UPC'].iloc[0]
                url_sugerido = df_comparar_filtrado[df_comparar_filtrado['Item'] == desc]['URL SKU'].iloc[0]
                precio_sugerido = df_comparar_filtrado[df_comparar_filtrado['Item'] == desc]['Final Price'].iloc[0]
                df_resultado.at[index, f'UPC Sugerido {i+1}'] = upc_sugerido
                df_resultado.at[index, f'Descripción de producto Sugerido {i+1}'] = desc
                df_resultado.at[index, f'Puntuación {i+1}'] = score
                df_resultado.at[index, f'URL SKU Sugerido {i+1}'] = fila_sugerida['URL SKU']
                df_resultado.at[index, f'Imagen Sugerida {i+1}'] = fila_sugerida['Image']
                df_resultado.at[index, f'Precio Final Sugerido {i+1}'] = fila_sugerida['Final Price']

    # Agregar los resultados del Store ID actual al DataFrame final
    df_resultado_final = pd.concat([df_resultado_final, df_resultado], ignore_index=True)

df_resultado_final
# Al final tendrás todos los resultados en df_resultado_final


In [ ]:
archivo_salida = final_file_name
df_resultado_final.to_csv(archivo_salida, index=False, encoding="utf-8-sig")